In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import geocoder
from sklearn.decomposition import PCA
import time
import random
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.metrics import roc_curve,auc
from sklearn import grid_search
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

~Assistant Function~

In [ ]:
#categorize number of people to groups
def peoples_to_catg(i):
    if (i==1.0)|(i==9.0):
        return 1
    if (2.0<=i<=4.0)|(i==8.0):
        return 2
    if 5.0<=i<=7.0:
        return 3

# categorize number of children to groups
def children_to_catg(i):
    if 0<=i <=3:
        return 1
    if 4<=i<=8:
        return 2
    if 9<=i:
        return 3

# convert ordinal categorical variables to order number
def ordinal_to_numeric(x):
    if x =='low':
        return 1
    if x=='medium':
        return 2
    if x=='high':
        return 3

# convert categorical variables to binary ones using one-hot
def onehot(df,columns=[]):
    for i in columns:
        dummies=[x for x in list(df[i].unique()) if str(x) != 'nan']
        for dummy in dummies:
            df[i+'.'+str(dummy)]=(df[i]==dummy).astype(int)
    df=df.drop(columns,axis=1)
    return df

## 1. Data Processing

### (1) Data Input

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
All = pd.concat((train, test))
print('Original Feature set:', All.columns)
print('length of train set:', len(train))
print('length of test set:', len(test))
print('length of all data:', len(All))

Length of all data: **9990**, Training set: **7578**, Hold-out set (with out target label):**2412**

### (2) Remove Abnormal Value in Target Variable

In [ ]:
print('Target varibale "cancel" contains:', list(train.cancel.unique()))
print('length of abnormal value in "cancel": ', len(train[train.cancel==-1]))
train=train[train.cancel!=-1]

Target varibale "cancel" contains: [0, 1, -1] 
<br>length of abnormal value in "cancel":  25

### (3) Convert Outlier

In [ ]:
print('******************convert outlier****************************')
# 'feature A should be limited to 100, so all observations with value of A >100 are capped as 100
train.loc[train['ni.age']>=100,'ni.age'] = 100
test.loc[test['ni.age']>=100,'ni.age'] = 100

### (4) Remove Adnormal Value in Target 

In [ ]:
print('******************adnormal value in target variable*************')
train=train[train.cancel!=-1]

### (5) Missing Value Treatment

In [ ]:
print('******************review missing values**********************')
print('train: \n',train.isnull().sum())
print('\n')
print('test: \n',test.isnull().sum())
print('*************************************************************')

In [ ]:
# import missing values in categorical variables with mode based on all data
train['B'] = train['B'].fillna(value=All['B'].mode())
test['B'] = test['B'].fillna(value=All['B'].mode())

# import missing values in numeric variables with mean based on all data
train['C']=train['C'].fillna(value=All['C'].mean())
test['C']=test['C'].fillna(value=All['C'].mean())

# remove missing value casue feature D only have one missing value in training dataset
train['D']=train[train['D'].notnull()]

## 2. Feature Extraction and Enginnering

### (1) Geocoding

In [ ]:
print('******************Geocoding************************************')
for i in train.index:
    print(i)
    try:
        g=geocoder.arcgis(str(int(train.loc[i,'zip.code'])))
        train.loc[i,'zip.city']=g.address.split(', ')[1] 
        train.loc[i,'zip.state']=g.address.split(', ')[2]
        train.loc[i,'zip.lat']=g.lat
        train.loc[i,'zip.lon']=g.lng
    except:
        train.loc[i,'zip.city']='Other' 
        train.loc[i,'zip.state']='Other'
        train.loc[i,'zip.lat']=0.0
        train.loc[i,'zip.lon']=0.0
    time.sleep(0.3)

for i in test.index:
    print(i)
    try:
        g=geocoder.arcgis(str(int(test.loc[i,'zip.code'])))
        test.loc[i,'zip.city']=g.address.split(', ')[1] 
        test.loc[i,'zip.state']=g.address.split(', ')[2]
        test.loc[i,'zip.lat']=g.lat
        test.loc[i,'zip.lon']=g.lng
    except:
        test.loc[i,'zip.city']='Other' 
        test.loc[i,'zip.state']='Other'
        test.loc[i,'zip.lat']=0.0
        test.loc[i,'zip.lon']=0.0
    time.sleep(0.3)

In [ ]:
print(len(train['zip.city'].unique()))
print(len(train['zip.state'].unique()))
print(len(test['zip.city'].unique()))
print(len(test['zip.state'].unique()))

There are 7 states, 125 cities in both of training and holdout dataset

### (2) rank city and zipcode based on average house price

In [ ]:
print('********rank city and zipcode based on market info*******************')
Train_rank=pd.read_csv('Train_price_rank.csv')
Train_rank=Train_rank.drop(['zip.state','zip.city'],axis=1)
Test_rank=pd.read_csv('Test_price_rank.csv')
Test_rank=Test_rank.drop(['zip.state','zip.city'],axis=1)
train=train.merge(Train_rank, how='left', on='zip.code')
test=test.merge(Test_rank, how='left', on='zip.code')

values = {'state_median_rank':0,'state_per_sqft_rank':0,'city_median_rank':0,'city_per_sqft_rank':0,'code_median_rank':0,'code_per_sqft_rank':0}
train=train.fillna(value=values)
test=test.fillna(value=values)

### (3) Feature Transforamtion

In [ ]:
# stanardizaion
train['D'] = (train['D']-train['D'].mean())/train['B'].std()
test['D'] = (test['D']-train['D'].mean())/test['B'].std()

# cluster (numberic - categorical)
train['B_cat'] = train['B'].apply(peoples_to_catg)
test['B_cat'] = test['B'].apply(peoples_to_catg)

# round value to int (some features like year, number of people, should not have decimal number)
train['E_round'] = round(train['E'],0)
test['E_round'] = round(test['E'],0)

# log transformation
train['I_log']=np.log10(train['I'])
test['I_log']=np.log10(test['I'])

# perform onehot on categorical variables
train=onehot(train,columns=['F','G','H'])
test=onehot(test,columns=['F','G','H'])


### (4) Feature Engineering

In [ ]:
# New features are generated based on business or mathmatic relationship between original features.
# Only code examples are shown as follows
train['A+B'] = train['A'] + train['B']
train['A/B'] = train['A']/train['B']
# create interaction features
train['A*B'] = train['A'].values*train['B'].values

## 3. Feature Selection

   1. Removed features based on improvement of model performance and feature importance.<br>
   2. Evaluate the selected features using monte carlo cross validation

### (1) Data Preparation

In [ ]:
original_features = train.columns
removed_features = ['feature list' ]
selected_features = [f for f in orginal_features if f not in removed_features]
X_train = train[selected_features]
y_train = train['cancel'].values
X_test = test[selected_features]

### (2) Fit in Logistic Regression

In [ ]:
# find best parameter C and AUC Score
LRCV=LogisticRegressionCV(Cs=[10**i for i in range(-5,4)],cv=10,n_jobs=4,solver='liblinear',scoring='roc_auc',penalty='l1',random_state=7)
LRCV.fit(X_train,y_train)
print('score for 10 Cs:',LRCV.scores_[1].mean(axis=0))
print('Best Score:',LRCV.scores_[1].mean(axis=0).max())
lst=list(LRCV.scores_[1].mean(axis=0))
index=lst.index(LRCV.scores_[1].mean(axis=0).max())
print('All chosen Cs:', LRCV.Cs_)
print('C for Best Score:', LRCV.Cs_[index])

### (3) Fit in XGBoost 

In [ ]:
# find best parameters and AUC Score
param_grid = {   
             'n_estimators': [300],
             'learning_rate': [0.02],
             'max_depth':[1],
             'min_child_weight':[1],
             'gamma':[0.0],
             'subsample':[0.6],
             'colsample_bytree':[0.2],
             'reg_alpha':[0.13]
             'reg_lambda':[0.005]
   } # create parameter space to search

start_time = time.time()
xgb = XGBClassifier(
        seed=7,
        verbose=100,
        objective='binary:logistic',
        booster='gbtree',
        n_estimators = 1500,
        learning_rate = 0.02,
        max_depth = 5,
        min_child_weight = 2,
        gamma = 0.0,
        subsample = 0.6,
        colsample_bytree = 0.2,
        reg_alpha = 0.13
        reg_lambda = 0.005        
        
        )

model = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=10, verbose=20,n_jobs=8, scoring='roc_auc')
model.fit(X_train,y_train)
print('--- Grid Search Completed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
print('Param grid:')
print(param_grid)
print('Best Params:')
print(model.best_params_)
print('Best CV Score:')
print(model.best_score_)

In [ ]:
# check feature importance
xgb = XGBClassifier(
        seed=7,
        verbose=100,
        objective='binary:logistic',
        booster='gbtree',
        n_estimators = 1500,
        learning_rate = 0.02,
        max_depth = 5,
        min_child_weight = 2,
        gamma = 0.0,
        subsample = 0.6,
        colsample_bytree = 0.2,
        reg_alpha = 0.13
        reg_lambda = 0.005        
        )

xgb.fit(X_train,y_train)

f_impor = pd.DataFrame(data={'features':selected_featureas,'importance':xgb.feature_importances_})
f_impor = f_impor.sort_values(by='importance')

f_impor.plot.bar(x='features',y='importance')


### (4) Monte Carlo Cross-Validation

In [ ]:
LR=LogisticRegression(C=0.5,penalty='l1',random_state=7,solver='liblinear', n_jobs=4)
col_name = X_train.colums



mccvdf = pd.DataFrame()

for i in col_name:
    mccvdf.loc[:,i] = []


def MCCV(train,label,seed):
    
    data_train, data_test, labels_train, labels_test = train_test_split(train, label, test_size=0.40, random_state=seed)
    LR.fit(data_train, labels_train)
    coefList = LR.coef_.tolist()[0]
    
    return coefList
    
for i in range(0,2000):
    coefficient = MCCV(X_train, y_train, seed=i)
    record = pd.Series(coefficient, index=col_name)
    mccvdf = mccvdf.append(record, ignore_index=True)

for i in range(mccvdf.shape[1]):
    plt.hist(mccvdf.iloc[:,i].values,bins=50)
    plt.title(col_name[i])
    plt.show()

## 4. Model Tuning and Selection

### (1) Data Preparation

In [ ]:
original_features = train.columns
removed_features = [ 'feature list' ]
selected_features = [f for f in orginal_features if f not in removed_features]
X_train = train[selected_features]
y_train = train['cancel'].values
X_test = test[selected_features]

### (2) Logistic Regression Tuning

In [ ]:
LRCV=LogisticRegressionCV(Cs=[10**i for i in range(-5,4)],cv=10,n_jobs=4,solver='liblinear',scoring='roc_auc',penalty='l1',random_state=7)
LRCV.fit(X_train,y_train)
print('score for 10 Cs:',LRCV.scores_[1].mean(axis=0))
print('Best Score:',LRCV.scores_[1].mean(axis=0).max())
lst=list(LRCV.scores_[1].mean(axis=0))
index=lst.index(LRCV.scores_[1].mean(axis=0).max())
print('All chosen Cs:', LRCV.Cs_)
print('C for Best Score:', LRCV.Cs_[index])

### (3) GBM tuning

In [ ]:
param_grid = {
        'max_depth': [],
        'min_samples_split':[],
        'min_samples_leaf':[],
        'max_features':[],
        'subsample':[],
        'n_estimators':[],
        'learning_rate':[]
        
    }

start_time = time.time()
clf = GradientBoostingClassifier(
    min_samples_split=75, #1% of 7552
    min_samples_leaf=7, # 10% of min_sample_split
    max_depth=5, # choose a number between 5 and 8
    max_features='sqrt', # tumb of rule
    subsample=0.8,
    n_estimators=3000,
    learning_rate=0.005,
    random_state=7)

model = GridSearchCV(estimator=clf, param_grid=param_grid, cv=10, verbose=20, scoring='roc_auc')
model.fit(X, y)
print('--- Grid Search Completed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
print('Param grid:')
print(param_grid)
print('Best Params:')
print(model.best_params_)
print('Best CV Score:')
print(model.best_score_)

### (4) SVM

In [ ]:
params={
    'C': [], 
    'gamma': []
}
model = GridSearchCV(SVC(kernel='rbf',probability=True), param_grid=params, verbose=20, n_jobs=8, cv=10, scoring='roc_auc')
model.fit(X,y)
print('Param grid:')
print(params)
print('Best Params:')
print(model.best_params_)
print('Best CV Score:')
print(-model.best_score_)

### (5) XGBoost

In [ ]:
param_grid = {   
             'n_estimators': [],
             'learning_rate': [],
             'max_depth':[],
             'min_child_weight':[],
             'gamma':[],
             'subsample':[],
             'colsample_bytree':[],
             'reg_alpha':[]
             'reg_lambda':[]
   } # create parameter space to search

start_time = time.time()
xgb = XGBClassifier(
        seed=7,
        verbose=100,
        objective='binary:logistic',
        booster='gbtree',
        n_estimators = 1500,
        learning_rate = 0.02,
        max_depth = 5,
        min_child_weight = 2,
        gamma = 0.0,
        subsample = 0.6,
        colsample_bytree = 0.2,
        reg_alpha = 0.13
        reg_lambda = 0.005        
        
        )

model = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=10, verbose=20,n_jobs=8, scoring='roc_auc')
model.fit(X_train,y_train)
print('--- Grid Search Completed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
print('Param grid:')
print(param_grid)
print('Best Params:')
print(model.best_params_)
print('Best CV Score:')
print(model.best_score_)

## 5. Model Ensemble

Provide Model Ensemble Framework

In [ ]:
start_time = time.time()
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models # n base models at the first level


        
    def fit_predict(self, X, y, T, seed): # y_pred = ensemble.fit_predict(X=X_train, y=y_train, T=X_test)
        X = np.array(X) # X_train
        y = np.array(y) # y_train
        T = np.array(T) # X_test

        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True,random_state=seed)) # generate index for kfolds

        S_train = np.zeros((X.shape[0], len(self.base_models))) # create N_train_rows* n_models array
        S_test = np.zeros((T.shape[0], len(self.base_models))) # create N_test_rows* n_models array

        for i, clf in enumerate(self.base_models): # fit the i th base model

            print('Fitting For Base Model #{0} / {1} ---'.format(i+1, len(self.base_models)))

            S_test_i = np.zeros((T.shape[0], len(folds))) # create N_test_rows* n_folds array

            for j, (train_idx, test_idx) in enumerate(folds): # use the i th base model to fit the j th fold

                print('--- Fitting For Fold #{0} / {1} ---'.format(j+1, self.n_folds))

                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]

                clf.fit(X_train, y_train) 
                y_pred = clf.predict_proba(X_holdout)[:,1] #get prediction from the j th fold
                S_train[test_idx, i] = y_pred # put into S_train
                S_test_i[:, j] = clf.predict_proba(T)[:,1] # put into S_test

                print('Elapsed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

            S_test[:, i] = S_test_i.mean(1) 

            print('Elapsed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

        print('--- Base Models Trained: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

        param_grid = {
             'n_estimators': [1000],
             'learning_rate': [0.002],
             'max_depth':[2],
             'min_child_weight':[19],
             'gamma':[0],
             'subsample':[0.5],
             'colsample_bytree':[0.5],
             'reg_alpha':[0.003]
             #'reg_lambda':[0.005]
        }
        grid = grid_search.GridSearchCV(estimator=self.stacker, param_grid=param_grid, cv=10, verbose=200, scoring='roc_auc')
        grid.fit(S_train, y)

        #message = 'to determine local CV score of #28'

        try:
            print('Param grid:')
            print(param_grid)
            print('Best Params:')
            print(grid.best_params_)
            print('Best CV Score:')
            print(-grid.best_score_)
            print('Best estimator:')
            print(grid.best_estimator_)
            print(message)
        except:
            pass

        print('--- Stacker Trained: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

        y_pred = grid.predict_proba(S_test)[:,1]

        return y_pred

print('--- Features Set: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
print('Number of Features: ', len(X_train.columns.tolist()))

# Please input the tuned parameters for eahc model

base_models = [
        GradientBoostingClassifier(
            n_estimators=,
            learning_rate=,
            random_state=, verbose=,
            min_samples_split=, 
            min_samples_leaf=, 
            max_depth=, 
            max_features=,
            subsample=
        ),
        XGBClassifier(
            seed=,verbose=,
            learning_rate=,
            n_estimators=, 
            max_depth=,
            min_child_weight = ,
            gamma = ,
            subsample=,
            colsample_bytree=,
            reg_alpha =  
        ),
        LogisticRegression(
            C=,verbose=,
            penalty=,
            random_state=,solver=
        ),
        SVC(
            probability=,
            C=,
            gamma=,
            random_state=)
    ]

ensemble = Ensemble(
        n_folds=,
        stacker=XGBClassifier(
            random_state=, verbose=
        ),
        base_models=base_models
    )

Random_avg = pd.DataFrame()
for i in range(20): 
    y_pred = ensemble.fit_predict(X=X_train, y=y_train, T=X_test, seed=i)
    Random_avg['y_predict'+str(i)]=y_pred
print('--- Submission Generated: %s minutes ---' % round(((time.time() - start_time) / 60), 2))